<a href="https://colab.research.google.com/github/Praveen76/Introduction-to-OpenMP/blob/main/Introduction_to_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Learning Objectives

At the end of the experiment, you will be able to:

* understand the parallelization in python
* implement multiprocessing using OpenMP

## Information

OpenMP is an Application Program Interface (API), jointly defined by a group of major computer hardware and software vendors. It provides a portable, scalable model for developers of shared memory parallel applications

OpenMP is the easiest to use and requires the minimum learning overhead and  most key parallel design patterns can be learned with OpenMP.

**Concurrency:** A condition of a system in which multiple
tasks are logically active at one time.

**Parallelism:** A condition of a system in which multiple
tasks are actually active at one time.

![link text](https://cdn.iisc.talentsprint.com/CDS/Images/OpenMP_Concurrent_Parallel.JPG)

### Setup Steps:

#### Importing required packages

In [3]:
import concurrent.futures
# The concurrent.futures module provides a high-level interface for asynchronously executing callables.
import time
from multiprocessing import Pool
# Pool is for parallelizing the execution of a function across multiple input values
import numpy as np

Let us derive a function that performs an action with sleep time and execute it with concurrent pool executors

### 1. Concurrency: Let's say a juggler is juggling 5 balls, but he is juggling only 1 ball at a time.

### 2. Multi-processing: Let's say multiple jugglers are juggling multiple balls at a time. So, each one of these jugglers is juggling seperate set of balls.

In [ ]:
start = time.perf_counter() # return time as nanoseconds

# function to delay the execution of a program
def do_something(seconds):
    print('Sleeping {} second(s)...'.format(seconds))
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'

# chops iterables into a number of chunks which it submits to the pool as separate tasks.
with concurrent.futures.ProcessPoolExecutor() as executor:
    secs = [5, 4, 3, 2, 1]
    results = executor.map(do_something, secs)

finish = time.perf_counter()
print('Finished in {} second(s)'.format(finish-start))

### Amdahl's Law: There's a limit to achieve speedup by increasing the number of cores.

In [1]:
p=0.9
n=64
def amdahls(p, n):
  return ((1-p)+p/n)**-1
print(amdahls(p, n))

8.767123287671234


### Parallelization in Python

Python does not thread very well. Specifically, Python has a very nasty drawback known as a Global Interpreter Lock (GIL). The GIL ensures that only one compute thread can run at a time. This makes multithreaded processing very difficult. Instead, the best way to go about doing things is to use multiple independent processes to perform the computations. This method skips the GIL, as each individual process has it’s own GIL that does not block the others. This is typically done using the multiprocessing module.

The pool object gives us a set of parallel workers we can use to parallelize our calculations. In particular, there is a map function (with identical syntax to the map() function used earlier), that runs a workflow in parallel.

Let’s try map() out with a test function that just runs sleep.

In [ ]:
# function for time sleep with 0.1 sec
def sleeping(arg):
    time.sleep(0.1)

%timeit list(map(sleeping, range(24)))

%timeit is an ipython magic function, which can be used to time a particular piece of code (A single execution statement, or a single method).

To know more about `%timeit` click [here](https://ipython.org/ipython-doc/dev/interactive/magics.html#magic-timeit)

Now let’s try it in parallel:

In [ ]:
pool = Pool(4)

In [ ]:
%timeit pool.map(sleeping, range(24))

The multiprocessing module has a major limitation: it only accepts certain  functions, and in certain situations. For instance any class methods, lambdas, or functions defined in __main__ won't work. This is due to the way Python “pickles” (read: serializes) data and sends it to the worker processes. “Pickling” simply can’t handle a lot of different types of Python objects.

Fortunately, there is a fork of the multiprocessing module called *multiprocess* that works just fine. *multiprocess* uses dill instead of pickle to serialize Python objects (read: send your data and functions to the Python workers), and does not suffer the same issues. Usage is identical:

In [4]:
!pip install multiprocess
from multiprocess import Pool

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00


In [ ]:
# shut down the old workers
pool.close()

pool = Pool(4)
%timeit pool.map(lambda x: time.sleep(0.1), range(24))
pool.close()

This is a general purpose parallelization recipe that we can use for your Python projects.

In [ ]:
# function to square the number
def square(x):
    return x**2

In [ ]:
# make sure to always use multiprocess

number_of_cores = 4
# start your parallel workers at the beginning of your script
pool = Pool(number_of_cores)

start = time.perf_counter()

# execute a computation(s) in parallel
result = pool.map(square, range(24))
result2 = pool.map(square, range(24))

finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')

# turn off your parallel workers at the end of your script
pool.close()

### MultiProcessing in Python using openMP

#### OpenMP
OpenMP employs a few principles in its programming model. The first is that everything takes place in threads. The second is the fork-join model, which comprises parallel regions in which one or more threads can be used

![link text](https://cdn.iisc.talentsprint.com/CDS/Images/Fork_join.png)

Above figure depicts the illustration of the fork–join paradigm, in which three regions of the program permit parallel execution of the variously colored blocks. Sequential execution is displayed on the top, while its equivalent fork–join execution is on the bottom.

#### Pymp
Because the goal of Pymp is to bring OpenMP-like functionality to Python, Pymp and Python should naturally share some concepts. A single master thread forks into multiple threads, sharing data and then synchronizing (joining) and destroying the threads.

As with OpenMP applications, when Pymp Python code hits a parallel region, processes – termed child processes – are forked and are in a state that is nearly the same as the “master process.” Note that these are forked processes and not threads, as is typical with OpenMP applications. As for the shared memory, according to the Pymp website, “… the memory is not copied, but referenced. Only when a process writes into a part of the memory [does] it gets its own copy of the corresponding memory region. This keeps the processing overhead low (but of course not as low as for OpenMP threads).”

In [5]:
# install the pymp
!pip -qq install pymp-pypi

To keep things simple, this is a serial code with a single array.

In [6]:
# creating an array of zeros
ex_array = np.zeros((100,), dtype='uint8')
for index in range(0, 100):
    # assigning 1
    ex_array[index] = 1
    print('Yay! {} done!'.format(index))

Yay! 0 done!
Yay! 1 done!
Yay! 2 done!
Yay! 3 done!
Yay! 4 done!
Yay! 5 done!
Yay! 6 done!
Yay! 7 done!
Yay! 8 done!
Yay! 9 done!
Yay! 10 done!
Yay! 11 done!
Yay! 12 done!
Yay! 13 done!
Yay! 14 done!
Yay! 15 done!
Yay! 16 done!
Yay! 17 done!
Yay! 18 done!
Yay! 19 done!
Yay! 20 done!
Yay! 21 done!
Yay! 22 done!
Yay! 23 done!
Yay! 24 done!
Yay! 25 done!
Yay! 26 done!
Yay! 27 done!
Yay! 28 done!
Yay! 29 done!
Yay! 30 done!
Yay! 31 done!
Yay! 32 done!
Yay! 33 done!
Yay! 34 done!
Yay! 35 done!
Yay! 36 done!
Yay! 37 done!
Yay! 38 done!
Yay! 39 done!
Yay! 40 done!
Yay! 41 done!
Yay! 42 done!
Yay! 43 done!
Yay! 44 done!
Yay! 45 done!
Yay! 46 done!
Yay! 47 done!
Yay! 48 done!
Yay! 49 done!
Yay! 50 done!
Yay! 51 done!
Yay! 52 done!
Yay! 53 done!
Yay! 54 done!
Yay! 55 done!
Yay! 56 done!
Yay! 57 done!
Yay! 58 done!
Yay! 59 done!
Yay! 60 done!
Yay! 61 done!
Yay! 62 done!
Yay! 63 done!
Yay! 64 done!
Yay! 65 done!
Yay! 66 done!
Yay! 67 done!
Yay! 68 done!
Yay! 69 done!
Yay! 70 done!
Yay! 71 done!
Ya

Let's start with Pymp version of the same code by importing the pymp

In [7]:
import pymp

In [8]:
# Below code will create 4 processes. Both tasks parallel process creation and joining will be handled by Context manager.

ex_array = pymp.shared.array((100,), dtype='uint8')
with pymp.Parallel(4) as p:
    for index in p.range(0, 100):
        ex_array[index] = 1
        # The parallel print function takes care of asynchronous output.
        p.print('Yay! {} done!'.format(index))

Yay! 25 done!
Yay! 26 done!
Yay! 27 done!
Yay! 75 done!
Yay! 76 done!
Yay! 77 done!
Yay! 78 done!
Yay! 79 done!
Yay! 80 done!
Yay! 81 done!
Yay! 0 done!
Yay! 1 done!
Yay! 2 done!
Yay! 3 done!
Yay! 4 done!
Yay! 5 done!
Yay! 6 done!
Yay! 50 done!
Yay! 51 done!
Yay! 52 done!
Yay! 53 done!
Yay! 54 done!
Yay! 82 done!
Yay! 7 done!
Yay! 8 done!
Yay! 9 done!
Yay! 10 done!
Yay! 11 done!
Yay! 55 done!
Yay! 56 done!
Yay! 57 done!
Yay! 12 done!
Yay! 13 done!
Yay! 28 done!
Yay! 29 done!
Yay! 30 done!
Yay! 31 done!
Yay! 32 done!
Yay! 33 done!
Yay! 34 done!
Yay! 35 done!
Yay! 36 done!
Yay! 37 done!
Yay! 38 done!
Yay! 39 done!
Yay! 40 done!
Yay! 41 done!
Yay! 83 done!
Yay! 14 done!
Yay! 58 done!
Yay! 59 done!
Yay! 60 done!
Yay! 15 done!
Yay! 16 done!
Yay! 17 done!
Yay! 18 done!
Yay! 42 done!
Yay! 43 done!
Yay! 61 done!
Yay! 62 done!
Yay! 63 done!
Yay! 44 done!
Yay! 45 done!
Yay! 46 done!
Yay! 47 done!
Yay! 19 done!
Yay! 20 done!
Yay! 21 done!
Yay! 22 done!
Yay! 23 done!
Yay! 24 done!
Yay! 48 done!
Ya

In [9]:
import pymp

# Create an array to store results
result_array = pymp.shared.array((100,), dtype='uint8')

# Parallel processing using pymp
with pymp.Parallel(4) as p:
    for index in p.range(0, 100):
        result_array[index] = index * index

# Output the results
print(result_array[:])


<ipython-input-9-cb2dbe2e20c4>:9: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 625 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  result_array[index] = index * index
<ipython-input-9-cb2dbe2e20c4>:9: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 676 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  result_array[index] = index * index
<ipython-input-9-cb2dbe2e20c4>:9: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 2500 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (th

[  0   1   4   9  16  25  36  49  64  81 100 121 144 169 196 225   0  33
  68 105 144 185 228  17  64 113 164 217  16  73 132 193   0  65 132 201
  16  89 164 241  64 145 228  57 144 233  68 161   0  97 196  41 144 249
 100 209  64 177  36 153  16 137   4 129   0 129   4 137  16 153  36 177
  64 209 100 249 144  41 196  97   0 161  68 233 144  57 228 145  64 241
 164  89  16 201 132  65   0 193 132  73]


#### OpenMP variables

Every parallel context provides its number of threads and the current thread's thread_num in the same way OpenMP does:

In [10]:

with pymp.Parallel(4) as p:
    p.print(p.num_threads, p.thread_num)

4 1
4 2
4 0
4 3


The original thread entering the parallel context always has `thread_num` 0

#### Variable scopes

The only implemented variable scopes are first private, shared and private.

- All variables that are declared before the `pymp.Parallel` call are implicitly first private
- All variables from the `pymp.shared` module are shared
- All variables created within a `pymp.Parallel` context are private.

The package `pymp.shared` provides a numpy array wrapper accepting the standard datatype strings, as well as shared list, dict, queue, lock and rlock objects wrapped from multiprocessing. High performance shared memory (ctypes) data structues are array, lock and rlock, the other data structures are synchronized via a *multiprocessing.Manager* and hence a little slower.

All data structures must be synchronized manually, if required, by using a lock. The parallel context offers one for your convenience:

In [11]:
# int array ; Create shared array
incremental_array = pymp.shared.array((1,), dtype='uint8')
print(incremental_array)
# list
no_of_threads = pymp.shared.list()

with pymp.Parallel(4) as p:
    for index in p.range(0, 100):
      # We want to block below piece of code, since we don't multiple threads executing in this section concurrently.
      # So, ony when one thread gets completed, and release the lock then only next thread will execute and complete.
        with p.lock:
            no_of_threads.append(p.thread_num)
            incremental_array[0] += 1
print(incremental_array)
print(no_of_threads)
# check the no.of threads
set([i for i in no_of_threads])

[0]
[100]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


{0, 1, 2, 3}

In [12]:
incremental_array

array([100], dtype=uint8)

#### Nested loops

When `pymp.config.nested is True`, it is possible to nest parallel contexts with the expected semantics:

**Uncomment the code below and execute the try except block again**

In [13]:
pymp.config.nested = True

In [14]:
# nested
try:
    with pymp.Parallel(4) as p1:
        with pymp.Parallel(2) as p2: # So each thread of 4 threads will fork ito 2 threads.
            p.print(p1.thread_num, p2.thread_num)
except:
    print("Its an Error!")

1 0
1 1
2 0
2 1
0 0
3 0
3 1
0 1


#### Laplace Solver Example

The common [Laplace solver](https://www.codeproject.com/Articles/1087025/Using-Python-to-Solve-Computational-Physics-Proble), is a little more detailed. The code is definitely not the most efficient, it uses loops

**Note:** Laplace solver is used as an example to calculate the computation time

In [15]:
nx = 1201
ny = 1201

# Solution and previous solution arrays
sol = np.zeros((nx,ny))

# make a copy of an array
soln = sol.copy()

for j in range(0,ny-1):
    sol[0,j] = 10.0
    sol[nx-1,j] = 1.0

for i in range(0,nx-1):
    sol[i,0] = 0.0
    sol[i,ny-1] = 0.0

In [16]:
# Iterate
start_time = time.perf_counter()
for kloop in range(1,10):
    soln = sol.copy()
    for i in range(1,nx-1):
        for j in range (1,ny-1):
            sol[i,j] = 0.25 * (soln[i,j-1] + soln[i,j+1] + soln[i-1,j] + soln[i+1,j])
end_time = time.perf_counter()

print('Elapsed wall clock time = %g seconds.' % (end_time-start_time) )

Elapsed wall clock time = 16.03 seconds.


Same Implementation of laplace solver using Pymp

In [17]:
# Solution and previous solution arrays
sol = pymp.shared.array((nx,ny))
soln = pymp.shared.array((nx,ny))

for j in range(0,ny-1):
    sol[0,j] = 10.0
    sol[nx-1,j] = 1.0

for i in range(0,nx-1):
    sol[i,0] = 0.0
    sol[i,ny-1] = 0.0

# Iterate
start_time = time.perf_counter()
with pymp.Parallel(4) as p:
    for kloop in range(1,10):
        soln = sol.copy()
        for i in p.range(1,nx-1):
            for j in p.range (1,ny-1):
                sol[i,j] = 0.25 * (soln[i,j-1] + soln[i,j+1] + soln[i-1,j] + soln[i+1,j])

end_time = time.perf_counter()
print('Elapsed wall clock time = %g seconds.' % (end_time-start_time) )

Elapsed wall clock time = 1.15186 seconds.
